Transaction validation

* Inputs are previously unspent
* Sum of inputs is greater than or equal to the outputs
* The SigScript unlocks the previous ScriptPubKey


In [3]:
# Example - is this transaction attempting to make money out of thin air?

from tx import Tx
from io import BytesIO
raw_tx = ('0100000001813f79011acb80925dfe69b3def355fe914bd1d96a3f5f71bf830\
3c6a989c7d1000000006b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccf\
cf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8\
e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278\
afeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88a\
c99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')
stream = BytesIO(bytes.fromhex(raw_tx))
transaction = Tx.parse(stream)
print(transaction.fee() >= 0)  # <1>

/Users/ds/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


True


ECDSA requires the public key P, the signature hash z, and the signature (r,s)

In [4]:
from ecc import S256Point, Signature
# SEC - standards for efficent cryptography - public key serialization
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e\
213bf016b278a')
# DER - distinguised encoding rules - for signatures
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031c\
cfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9\
c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
print(point.verify(z, signature))

True


Getting the signature hash

* Can't hash the transaction, since the signature itself is embedded in the SigScript
* Transaction is modified before signing - a signature hash for each input is computed

The process:

1. Empty all ScriptSigs before checking (or for creating, probably has empty ScriptSigs when creating).
2. Replace the SigScript of the transaction with ScriptPubKey of the previous transaction output
3. Append the hash type encoded little-endian over 4 bytes

The hash256 of the modified transaction can be taken, and the big-endian integer is taken as z

In [5]:
from helper import hash256
modified_tx = bytes.fromhex('0100000001813f79011acb80925dfe69b3def355fe914\
bd1d96a3f5f71bf8303c6a989c7d1000000001976a914a802fc56c704ce87c42d7c92eb75e7896\
bdc41ae88acfeffffff02a135ef01000000001976a914bc3b654dca7e56b04dca18f2566cdaf02\
e8d9ada88ac99c39800000000001976a9141c4bc762dd5423e332166702cb75f40df79fea1288a\
c1943060001000000')
h256 = hash256(modified_tx)
z = int.from_bytes(h256, 'big')
print(hex(z))


0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6


In [6]:
# Putting it all together

from ecc import S256Point, Signature
sec = bytes.fromhex('0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e\
213bf016b278a')
der = bytes.fromhex('3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031c\
cfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9\
c8e10615bed')
z = 0x27e0c5994dec7824e56dec6b2fcb342eb7cdb0d0957c2fce9882f715e85d81a6
point = S256Point.parse(sec)
signature = Signature.parse(der)
point.verify(z, signature)

True

In [7]:
from ecc import PrivateKey
from helper import hash256, little_endian_to_int
secret = little_endian_to_int(hash256(b"Nouvelle Methode complete de cornet a pistons et d'instruments a pistons par Alexandre Petit"))
private_key = PrivateKey(secret)
private_key.point.address(compressed=False, testnet=True)

'n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4'

In [9]:
# Fund the address
!curl -X POST --data '{"address": "n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4", "amount": 0.02}' http://localhost:3000/faucet

{"txId":"233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e"}


On the command line

```
alias bcli='docker exec -it bitcoin bitcoin-cli'        
ds@dss-MacBook-Pro ~ % bcli -regtest getnewaddress                     
bcrt1q6zdhs72upcwpuu5c85e2k7ncum7sjjn5fhrjty
```

Current UTX0 set

```
bcli gettransaction 233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e
{
  "amount": -0.02000000,
  "fee": -0.00000147,
  "confirmations": 1,
  "blockhash": "3ea2bcd72d603715e94b2c2ff84663d7652669ea46ad555aecf22737b4acce52",
  "blockheight": 102,
  "blockindex": 1,
  "blocktime": 1694098554,
  "txid": "233bd792c876693e6719716df22bef9e88f99634f4a3ef2ea91689057e9bfd4e",
  "wtxid": "1dcbe07b4024a1fcd0fe5a9e3df9d27f9e0bab5604baf582d2d8cfdf2d98d1a4",
  "walletconflicts": [
  ],
  "time": 1694098554,
  "timereceived": 1694098554,
  "bip125-replaceable": "no",
  "details": [
    {
      "address": "n3zzACo1BgNCDK928v6KpnWvnQ3KC9Aox4",
      "category": "send",
      "amount": -0.02000000,
      "vout": 1,
      "fee": -0.00000147,
      "abandoned": false
    }
  ],
  "hex": "0200000000010195688d7e1337f698dd452434257c543a748219669111544b4503fe1d4605a9ec0000000000fdffffff02ed6ce729010000001976a9140fa738fd6297380aacb498c74ac038e9db83009588ac80841e00000000001976a914f69ff8808047eeddf564911ae7e0384b2cacf76288ac02473044022001083413c63c76b93ff468f2ba693a042be14162902de83fcabbe79b32f33a340220632991f6226ebba1746c771f4ec82c835b7ae5735af6568374df0170f23c54c6012103efb2c883e93e2977c1341b8637d757ce47b970e02aa13acd65ca26f844de8d6400000000"
}
```

From the cli we can create another address

```
bcli -regtest getnewaddress                     
bcrt1q6zdhs72upcwpuu5c85e2k7ncum7sjjn5fhrjty
```

Can we send from the address controlled by our private key to the new address, receiving change back to our original address (not a recommended practive, but for simplicity)?